We load the data with the features.
The airfare in euro is the target value.
We hot encode the categorical feature cabin_class and drop one.
To tune the regressor, we divide the data into training, testing and validation sets.
I ended up not using the distance_code feature since when I tried to encode it, it took too much space and I couldn't continu.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict   
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

df = pd.read_csv('features_regression.csv')
df = df.drop(['search_id'], axis=1) 
#df = df.values
y = df['fare_eur'].values
X = df.drop('fare_eur',1)
dummies = pd.get_dummies(X[['cabin_class']])
X = pd.concat([X, dummies], axis=1)
X = X.drop(['cabin_class','cabin_class_mixed','distance_code'], 1)
#%%
X_, X_test, y_, y_test = train_test_split(X,y, test_size=0.40, random_state=42)#[:20000] 
X_train, X_valid, y_train, y_valid = train_test_split(X_ ,y_ , test_size=0.40, random_state=42)


I chose to use a random forest regressor, because it is a powerful algorithm that is easy to tune.
There are only two parameters, the number of features considered at each level of the tree (for regression, in auto mode considering all the features is recommended) and the number of tree to build.
Since the results improve with a growing number of trees, I only looked at a model for which the calculation time is acceptable. 
First I wanted to use a cross validation algorithm to see the performance of the model but due to the processing time, I didn't and simply looked at some metrics on the testing set.
I also plotted a figure to see the importance of each features and it is the partner_id that has the most influence.

In [ ]:
n_estimators = [10,50,100,150]
metric = []
for n_estimator in n_estimators:
    forest = RandomForestRegressor(n_estimator, max_features='auto')
    forest.fit(X_train,y_train)
    scores = forest.predict(X_test)#cross_val_predict(forest, X_train, y_train, cv=10)
    metric.append(metrics.mean_squared_error(y_test, scores))
    print(metrics.mean_squared_error(y_test, scores))
    
    forest.fit(X_train, y_train)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X_train.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X_train.shape[1]), indices)
    plt.xlim([-1, X_train.shape[1]])
    plt.show()
print(metric)

I chose 100 trees as the best number of trees to build the model because the processing time was acceptable.
To be sure that I don't tune in too much the model, I predicted some data that the model has never see.

In [ ]:
best_n_estimator = 100
forest_model = RandomForestRegressor(n_estimator, max_features='auto')
forest_model.fit(X_train,y_train)
        
predicted_valid = forest_model.predict(X_valid)
print("Regression problem, metrics on validation set:")
print(metrics.classification_report(y_valid, predicted_valid))

Conclusion:
I was not satisfied with my feature that accounted for the distance so I would work more on that.
The fact that the utc offset was not taken into account in the datetimes is also a thing to improve.
Overall, I would work more on the features, try to find more meaningful features for the airfare prediction businesswise and look at the columns that I didn't consider.
I might also look at the influence of features on the model to go back and forth when engineering the features.